In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [11]:
!unzip -q train.zip -d .

In [12]:
import os
print( len( os.listdir('/content/train/') ) )

25000


In [ ]:
from keras.utils.data_utils import shutil
import os

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/'+i, '/content/dataset/cat/'+i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/'+i, '/content/dataset/dog/'+i )

print( len( os.listdir('/content/dataset/cat/') ) )
print( len( os.listdir('/content/dataset/dog/') ) )


In [17]:
%rm -rf dataset

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt


# train data 전체 데이터의 80%
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset = 'training',
    validation_split=0.2,
    seed=1234
)

# validation data 전체 데이터의 20%
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset = 'validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)

def prefunc(i, 답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 답

train_ds = train_ds.map(prefunc)
val_ds = val_ds.map(prefunc)


for i, 정답 in train_ds.take(1):
  print(i)
  print(정답)
  #plt.imshow( i[0].numpy().astype('uint8') )
  #plt.show()

In [26]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), padding="same", activation='relu', input_shape=(64,64,3)),  # color 사진이라 3
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), padding="same", activation='relu' ),  # color 사진이라 3
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Dropout(0.2),   # overfitting 완화하기 -> 윗 노드들중에 일부 제거 (예시는 0.2 )

    tf.keras.layers.Conv2D(128,(3,3), padding="same", activation='relu' ),  # color 사진이라 3
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(1,activation="sigmoid"),  # 개, 고양이인지 판별이라 노드 갯수는 1
])

model.summary()

model.compile( loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'] )
model.fit(train_ds, validation_data=val_ds, epochs=5)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 128)      